In [ ]:
arrow::read_parquet(here("data", "atpTrans.parquet")) -> atp
atp %>% glimpse()

ERROR: Error in augment.data.frame(.): augment's first argument should be a model, not a data.frame


In [ ]:
atp %>% filter(tournament.location == "Australia") -> atpA

In [ ]:
# parquet doesn't seem to save factor
atpA %>% mutate(
    winner.backHand = winner.backHand             %>% as.factor,
    winner.domHand  = winner.domHand              %>% as.factor,
    looser.backHand = looser.backHand             %>% as.factor,
    looser.domHand  = looser.domHand              %>% as.factor,
    match.tournamentRound = match.tournamentRound %>% as.factor,
    tournament.ground = tournament.ground         %>% as.factor,
    tournament.location = tournament.location     %>% as.factor,
    winner.bornAt = winner.bornAt                 %>% as.factor,
    looser.bornAt = looser.bornAt                 %>% as.factor,
    match.setsCount = match.setsCount             %>% as.factor,
) -> atpAF

In [ ]:
if (FALSE) {
    atpAF %>% SmartEDA::ExpReport(op_file = "temp.html")
}

In [ ]:
(atpAF %>% nrow() -> nmatches.inicial)

[1] 21388

In [ ]:
# things to fix:
# prize ✔
# maybe dont include those that are both heights missing✔
# remove rets walkovers and defs✔
# decide on how to remove best of 5 tournaments✔
# turn nas into a factor too in backhand and domHand (backhand already has an unkown)
# fazer eda do rank para perceber como tratalos
# tratar heights✔

In [ ]:
# prize
atpAF %>% 
    select(tournament.prize) %>%
    filter(str_detect(tournament.prize, fixed("$"), negate=T))

tournament.prize
<chr>


In [ ]:
# no prizes without dollar
atpAF %>%
    mutate(tp = str_remove_all(tournament.prize, r"{[,\$]}")) %>%
    filter(str_detect(tp, r"{[^0-9]}")) %>% 
    select(tp)

tp
<chr>
A9609870
A9609870
A9609870
A9609870
A9609870
A9609870
A9609870
A9609870
A9609870


In [ ]:
# australian dollar
atpAF %>%
    mutate(tp = str_remove_all(tournament.prize, r"{[,\$A]}")) %>%
    filter(str_detect(tp, r"{[^0-9]}")) %>% 
    select(tp)

tp
<chr>


In [ ]:
doll_to_aus <- \(x) x*1.48
atpAF %>%
    rename(tp = tournament.prize) %>% 
    mutate(tournament.prize = case_when(
        tp %>% is.na()                 ~ NA,
        tp %>% str_detect(fixed("A$")) ~ tp %>% str_remove_all(r"{[\A$,]}") %>% as.double(),
        tp %>% str_detect(fixed("$"))  ~ tp %>% str_remove_all(r"{[\$,]}")  %>% as.double() %>% doll_to_aus(),
        TRUE ~ 9e+9
     )) %>% select(-tp) -> atpAFP
atpAFP %>% filter(tournament.prize == 9e+9) %>% nrow()

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `tournament.prize = case_when(...)`.
Caused by warning in `doll_to_aus()`:
! NAs introduced by coercion"


[1] 0

In [ ]:
library(skimr)
# atpAFP %>% skim(winner.height, looser.height) # yields error because of jup

In [ ]:
atpAFP %>% mutate(
    winner.height = ifelse(winner.height == 0, NA, winner.height), 
    looser.height = ifelse(looser.height == 0, NA, looser.height), 
) -> atpAFPH.wNA
# atpAFPH.wNA %>% skim(winner.height, looser.height)
# interpolate heights?
# maybe if we don't get enough observations

In [ ]:
atpAFPH <- atpAFPH.wNA %>% filter(!is.na(winner.height) & !is.na(looser.height))
# atpAFPH %>% skim(winner.height, looser.height)

In [ ]:
atpAFPH %>% filter(winner.height < 160) %>% distinct(winner.id)

winner.id
<chr>
s406
h431


In [ ]:
atpAFPH %>% filter(looser.height < 160) %>% distinct(looser.id)

looser.id
<chr>
s406
h431


TODO texto mas

* h431 -> 180 cm
* s406 -> 188 cm

In [ ]:
atpAFPHF <- atpAFPH %>% mutate(winner.height = case_when(
    winner.id == "h431" ~ 180,
    winner.id == "s406" ~ 188,
    TRUE ~ winner.height
), looser.height = case_when(
    looser.id == "h431" ~ 180,
    looser.id == "s406" ~ 188,
    TRUE ~ looser.height
))
# atpAFPHF %>% skim(winner.height, looser.height)

In [ ]:
atpAFPHFR <- atpAFPHF %>% 
    filter(!match.wasRet, !match.wasWalkover, !match.wasDef) %>%
    select(-match.wasRet, -match.wasWalkover, -match.wasDef)
# atpAFPHFR %>% select(starts_with("match")) %>% skim()

In [ ]:
# tournaments best of 5
atpAFPHFR -> atp.wbo5
atp.wbo5 %>% 
    group_by(tournament.name) %>% 
    filter(!any(match.setsCount %>% as.integer > 3)) %>%
    ungroup() -> atp.wo45

In [ ]:
atp.wo45 %>% 
    group_by(tournament.name) %>% 
    filter(all(match.setsCount == 3)) %>% 
    summarise()

tournament.name
<chr>
AUS v. MAR WG PO
AUS v. SWE WG 1st RD
AUS vs. SVK WG Play-Off
Adelaide-2
ROM V AUS QF


TODO ver quais destes sao melhor de 3 e melhores de 5, dps tirar esses

In [ ]:
toRem = c("AUS v. MAR WG PO","AUS v. SWE WG 1st RD","AUS vs. SVK WG Play-Off","ROM V AUS QF")
atp.wo45 %>% filter(!(tournament.name %in% toRem)) -> atp.wobo5

In [ ]:
atp.wobo5 %>% nrow()

[1] 6839

In [ ]:
# atp.wobo5 %>% skim(winner.domHand, looser.domHand, winner.backHand,looser.backHand)

In [ ]:
df <- atp.wobo5 %>% 
    filter(!(looser.domHand %>% is.na) & looser.domHand != "Ambidextrous") %>%
    filter(!(looser.rank %>% is.na) & !(winner.rank %>% is.na))
# df %>% skim(winner.domHand, looser.domHand, winner.backHand,looser.backHand)

In [ ]:
# features q faltam:
# diferenca de alturas✔
# media de alturas✔
# idades (*)
# numero de jogos jogados - diferenca/numero de wins/winrate
# frequencia de jogos do pais de naturalidade
# se pais tem jogos - condicao/contagem/diferenca/diferenca de condicao
# domhand comparacao ✔
# backhand comparacao ✔
# tipo do torneio (afonse)
# frequencia do torneio
# mes do torneio ✔
# distancia ao ultimo torneio/primeiro ✔
# premio (precisa eda) 
# distancia da ronda final (chaves)
# diferenca de rank ✔ /categoria de rank (eda)

In [ ]:
firstT <- df %>% arrange(tournament.date_start) %>% head(1) |> _$tournament.date_start
firstT 

[1] "1974-12-09"

In [ ]:
df %>%
    unite("match.domHands", c(winner.domHand, looser.domHand)) %>%
    unite("match.backHands", c(winner.backHand, looser.backHand)) %>%
    mutate(
        match.domHands          = match.domHands %>% as.factor(),
        match.backHands         = match.backHands %>% as.factor(),
        match.diffHeights       = winner.height - looser.height,
        match.avgHeights        = rowMeans(across(c(winner.height, looser.height))),
        tournament.month        = month(tournament.date_start, label=T),
        tournament.quarter      = quarter(tournament.date_start) %>% as.factor(),
        tournament.dateDistance = difftime(firstT, tournament.date_start, units = "weeks"),
        match.diffRank          = winner.rank - looser.rank,
        match.avgRank           = rowMeans(across(c(winner.rank, looser.rank)))
    ) %>% 
    select(-winner.name, -winner.id, -looser.name, -looser.id, -tournament.location) %>% 
    mutate(match.setsCount = fct_drop(match.setsCount))->  dfF
# dfF %>% skim()

In [ ]:
dfF %>% nrow()

[1] 6365

In [ ]:
# spliting
set.seed(1)
df.split <- initial_split(dfF, prop = 3/4, strata = match.setsCount)
df.train <- training(df.split)
df.test  <- testing (df.split)
# vfold
# df.10fold <- vfold_cv(dfF, v = 5, repeats = 5)

In [ ]:
# first model
models.first <- workflow() %>% 
    add_model(logistic_reg(mode="classification")) %>%
    add_formula(match.setsCount ~ 
        tournament.dateDistance + 
        match.domHands + 
        match.backHands + 
        match.tournamentRound + 
        tournament.ground + 
        tournament.quarter + 
        match.hadTieBreaker + 
        tournament.duration + 
        match.diffHeights + 
        match.diffRank,
    ) %>% fit(df.train) # fit_resamples(df.10fold)
models.first %>% glance()

null.deviance,df.null,logLik,AIC,BIC,deviance,df.residual,nobs
<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
6099.459,4772,-2910.795,5887.589,6101.124,5821.589,4740,4773


In [ ]:
models.first.pred <- predict(models.first, df.test)
models.first.vs <- cbind(pred = models.first.pred, truth = df.test$match.setsCount) %>% rename(pred = .pred_class)
models.first.vs 

pred,truth
<fct>,<fct>
2,2
2,3
2,2
3,3
2,3
2,2
2,2
2,2
2,3


In [ ]:
models.first.vs %>% accuracy(truth= truth, estimate = pred)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.673995


In [ ]:
models.first.vs %>% f_meas(truth= truth, estimate = pred)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
f_meas,binary,0.7831174


In [ ]:
models.first.vs %>% roc_auc_vec(truth= .$truth, estimate = .$pred, estimator = "binary")

ERROR: [1m[33mError[39m in `roc_auc_vec()`:[22m
[33m![39m `estimate` should be a numeric vector, not a `factor` vector.


In [ ]:
models.first.vs %>% roc_curve(truth= truth, estimate = pred, estimator = "binary")

ERROR: [1m[33mError[39m in `roc_curve()`:[22m
[1m[22m[33m![39m Can't rename variables in this context.
